Code to retrieve multi-domain proteins for analysis of domain to domain predicted aligned error.

In [1]:
import requests
import pandas as pd
import numpy as np
import random
import csv

First we take the list of PDB IDs with permanent domain-domain interactions as determined by [Sidhanta _et al._, 2023](https://doi.org/10.1002/prot.26581) and use that to build our list of UniProt IDs.

In [17]:
'''
Build the PDB Search API request
'''

# Set the search parameters
def get_uniprot_ids_for_pdbs(pdb_ids):
    uniprot_ids = []
    for pdb in pdb_ids:
        url = f"https://data.rcsb.org/rest/v1/core/uniprot/{pdb}/1"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            # Parse the response to extract UniProt IDs
            if 'rcsb_uniprot_container_identifiers' in data[0]:
                if 'uniprot_id' in data[0]['rcsb_uniprot_container_identifiers']:
                    uniprot_id = data[0]['rcsb_uniprot_container_identifiers']['uniprot_id']
                    uniprot_ids.append(uniprot_id)
        
        else:
            print(f"Failed to fetch data: {response.status_code}")

    return uniprot_ids


In [20]:
# Example usage


pdb_ids = []

with open ('./project_pipeline/data/sidhanta_pdbs.csv') as f:
    reader = csv.reader(f)
    next(reader, None)
    for row in reader:
        pdb_ids.append(''.join(row).upper())

print(pdb_ids)
uniprot_ids = get_uniprot_ids_for_pdbs(pdb_ids)
print(uniprot_ids)


['1A62', '1H2W', '1NTY', '1S6Y', '1Y02', '2J3X', '3E2Q', '3TXN', '5N4F', '1A8D', '1IG8', '1OHC', '1SAT', '1YDX', '2J8G', '3F1W', '4A2B', '5T5X', '1AOA', '1IH7', '1P4X', '1SQW', '1Z5V', '2NLK', '3FUQ', '4EQ3', '5UV2', '1CDY', '1IHG', '1PI6', '1T1E', '1ZAR', '2VAP', '3GON', '4FJQ', '5VF5', '1DHY', '1J5Y', '1Q7H', '1TXD', '1ZRL', '2VXR', '3H7I', '4FQZ', '6APE', '1EIF', '1K1S', '1Q8I', '1U3D', '2B3X', '2WMF', '3LNU', '4GQ4', '1EWF', '1KHB', '1QS2', '1U5P', '2D5B', '2WN4', '3MZF', '4HD9', '1F2Q', '1KJW', '1QTS', '1UCT', '2EF0', '2YHS', '3N7L', '4J12', '1F5N', '1L8Q', '1QZZ', '1UEK', '2FY2', '2YV1', '3OF1', '4LGN', '1F97', '1MD8', '1R2J', '1V0W', '2GNO', '2YZQ', '3SBS', '4O95', '1FVI', '1N4K', '1RHS', '1VCT', '2GNX', '3A0F', '3SQZ', '4XEH', '1G4R', '1NE9', '1RLR', '1W9H', '2I0K', '3BZ6', '3TEW', '5IL3', '1GV2', '1NR0', '1S5J', '1X6O', '2ILL', '3CE0', '3TRE', '5J4O']
['P0AG30', 'P23687', 'O75962', 'P84135', 'Q8WZ73', 'O69275', 'P09546', 'Q7KLV9', 'H2E7Q8', 'P04958', 'P04807', 'O60729', 'P2369

In [22]:
with open('./project_pipeline/data/sidhanta_uniprots.csv', 'w') as f:
    for uniprot in uniprot_ids:
        f.write(uniprot + '\n')

Now we get the domain information for each UniProt ID

In [23]:
'''
Commented out
'''
def get_domains(uniprot_id):
    '''
    Get the domain information from UniProtKB
    '''
    print(f'Getting domains for {uniprot_id}')
    url = f'https://rest.uniprot.org/uniprotkb/search?query=accession:{uniprot_id}&fields=ft_domain'
    response = requests.get(url)
    response_dic = response.json()
    domains = []
    try:
        features = response_dic['results'][0]['features']
        # Get the start and end of any domains
        for i in range(len(features)):
            if response_dic['results'][0]['features'][0]['type'] == 'Domain':
                start = str(features[i]['location']['start']['value'])
                end = str(features[i]['location']['end']['value'])
                domains.append((start + '-' + end))

        domains_string = ','.join(domains)

    except KeyError:
        print(f'No domains found for {uniprot_id}')
        domains_string = None

    return domains_string

# def single_domains(uniprots):
#     '''
#     Get the single domains from UniProtKB
#     '''
#     domains = {'uniprot': [], 'region': []}
#     # Get domains for the uniprot ids
#     for i in range(len(uniprots)):
#         uniprot_id = uniprots[i]
#         region = get_domains(uniprot_id)
#         domains['uniprot'].append(uniprot_id)
#         domains['region'].append(region)

#     # Convert to pandas dataframe
#     domains_df = pd.DataFrame.from_dict(domains, orient='columns')

#     return domains_df

In [25]:
domain_dict = {'uniprot': [], 'domains': []}

for uniprot in uniprot_ids:
    domains = get_domains(uniprot)
    domain_dict['uniprot'].append(uniprot)
    domain_dict['domains'].append(domains)

Getting domains for P0AG30
Getting domains for P23687
Getting domains for O75962
Getting domains for P84135
Getting domains for Q8WZ73
Getting domains for O69275
Getting domains for P09546
Getting domains for Q7KLV9
Getting domains for H2E7Q8
Getting domains for P04958
Getting domains for P04807
Getting domains for O60729
Getting domains for P23694
Getting domains for Q49434
Getting domains for P15057
Getting domains for P15873
Getting domains for Q9WZU0
Getting domains for P97784
Getting domains for P13797
Getting domains for Q38087
Getting domains for Q2G1N7
Getting domains for Q9Y221
Getting domains for P23258
Getting domains for P23470
Getting domains for A7GBG3
Getting domains for B4XN22
Getting domains for A0A1C9J6A7
Getting domains for P01730
Getting domains for P26882
Getting domains for P46680
Getting domains for Q8RR56
Getting domains for O30245
Getting domains for Q57816
Getting domains for Q8DR49
Getting domains for M4GGS0
Getting domains for A0A2R2JFS1
Getting domains for 

In [26]:
# Convert to dataframe
domain_df = pd.DataFrame.from_dict(domain_dict, orient='columns')
domain_df.head()

# Save to csv
# domain_df.to_csv('./project_pipeline/data/sidhanta_domains.csv', index=False)

,uniprot,domains
0,P0AG30,48-123
1,P23687,
2,O75962,"65-210,1292-1467,1480-1591,1656-1721,1969-2145..."
3,P84135,200-416
4,Q8WZ73,"101-120,250-264"


In [36]:
# remove any proteins that may overlap with my autoinhibited set
autoinhibited = pd.read_csv('./project_pipeline/data/classified_files_3.tsv', sep='\t').astype('object')
common = domain_df['uniprot'].isin(autoinhibited['uniprot'])
domains_df = domain_df.drop(domain_df[common].index).reset_index(drop=True)

# Save the dataframe
domains_df.to_csv('./project_pipeline/data/sidhanta_domains.csv', index=False)
domains_df.head()

,uniprot,domains
0,P0AG30,48-123
1,P23687,
2,O75962,"65-210,1292-1467,1480-1591,1656-1721,1969-2145..."
3,P84135,200-416
4,Q8WZ73,"101-120,250-264"


Second, we take the list of proteins defined as "multi-domain" by SCOPe on the PDB and filter those down

In [ ]:
'''
Build the PDB Search API request
'''

# Set the search parameters
url = 'https://search.rcsb.org/rcsbsearch/v2/query'
query = {
  "query": {
    "type": "group",
    "nodes": [
      {
        "type": "group",
        "logical_operator": "and",
        "nodes": [
          {
            "type": "terminal",
            "service": "text",
            "parameters": {
              "attribute": "rcsb_polymer_instance_annotation.annotation_lineage.id",
              "operator": "exact_match",
              "value": "56572",
              "negation": False
            }
          },
          {
            "type": "terminal",
            "service": "text",
            "parameters": {
              "attribute": "rcsb_polymer_instance_annotation.type",
              "operator": "exact_match",
              "value": "SCOP",
              "negation": False
            }
          }
        ],
        "label": "nested-attribute"
      },
    ],
    "logical_operator": "and",
    "label": "text"
  },
  "return_type": "polymer_entity",
  "request_options": {
    "group_by_return_type": "groups",
    "group_by": {
      "aggregation_method": "matching_uniprot_accession",
      "ranking_criteria_type": {
        "sort_by": "rcsb_entry_info.resolution_combined",
        "direction": "asc"
      }
    },
    "return_all_hits": True,
    "results_content_type": [
      "experimental"
    ],
    "sort": [
      {
        "sort_by": "score",
        "direction": "desc"
      },
      {
        "sort_by": "size",
        "direction": "desc"
      }
    ],
    "scoring_strategy": "combined"
  }
}

In [ ]:
response = requests.post(url, json=query)

response_dic = response.json()

# Get the PDB IDs
group_set = response_dic['group_set']

# Get the list of uniprot ids
uniprots = [group_set[i]['identifier'] for i in range(len(group_set))]


with open('./project_pipeline/data/multi_domain_uniprots.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['uniprot'])
    writer.writerows(uniprots)

In [ ]:
'''
Commented out
'''
# domains_df = domains_df.drop(domains_df[domains_df['region'] == ''].index).reset_index(drop=True)
# domains_df = domains_df.dropna().reset_index(drop=True)

# # Remove any proteins with only one annotated domain
# for i in range(len(domains_df)):
#     region = domains_df.loc[i, 'region']
#     count = region.count('-')
#     if count <= 1:
#         domains_df = domains_df.drop(i)

# # remove any proteins that may overlap with my autoinhibited set
# autoinhibited = pd.read_csv('./project_pipeline/data/classified_files_3.tsv', sep='\t').astype('object')
# common = domains_df['uniprot'].isin(autoinhibited['uniprot'])
# print(common)
# domains_df = domains_df.drop(domains_df[common].index).reset_index(drop=True)

# # Save the dataframe
# domains_df.to_csv('./project_pipeline/data/multi_domain_domains.csv', index=False)

In [2]:
autoinhibited = pd.read_csv('./project_pipeline/data/classified_files_3.tsv', sep='\t').astype('object')
single_df = pd.read_csv('./project_pipeline/data/single_domain_domains.csv')
common2 = single_df['uniprot'].isin(autoinhibited['uniprot'])

In [3]:
'''
How many multi-domain proteins have only two designated domains?
'''
md = pd.read_csv('./project_pipeline/data/multi_domain_domains.csv')

# Get the number of domains
md['num_domains'] = md['region'].str.count(',') + 1

# Get the number of proteins with only two domains
md_2 = md[md['num_domains'] == 2]
print(len(md_2))

61


In [3]:
'''
Select proteins with two domains. Label those domains 'region 1' and 'region 2' and pass the protein
through my predicted aligned error script.
'''
domains_df = pd.read_csv('./project_pipeline/data/multi_domain_domains.csv').astype('object')

domains_df['region'] = domains_df['region'].str.split(',')
two_domains_list = [row for index, row in domains_df.iterrows() if len(row['region']) == 2]
two_domains = pd.DataFrame(two_domains_list).reset_index(drop=True)

expand = pd.DataFrame(two_domains['region'].to_list(), columns=['region_1', 'region_2'])
expand = expand.join(two_domains['uniprot'], how='left')
expand = expand[['uniprot', 'region_1', 'region_2']]
expand.head()

expand.to_csv('./project_pipeline/data/multi_domain_regions.csv', index=False)

This next step assumes that scripts/control_domains_pae.py has been run. We make a file to be run in our snakemake multi-domain pipeline. 

In [37]:
# Take out proteins in multi_domain_regions.tsv that are not in multi_domain_pae.tsv
multi_domain_regions = pd.read_csv('./project_pipeline/data/multi_domain_regions.tsv', sep='\t')
multi_domain_pae = pd.read_csv('./project_pipeline/data/multi_domain_pae.tsv', sep='\t')

common = multi_domain_regions['uniprot'].isin(multi_domain_pae['uniprot'])
multi_domain_regions = multi_domain_regions[common]
multi_domain_regions.to_csv('./project_pipeline/data/multi_domain.tsv', sep='\t', index=False)